In [2]:
!pip install plotly

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings("ignore")


In [3]:
# M1 Data integration
file_path = '/Users/shadeer/Desktop/HOME/AI/PulseID/Notebook-instance/Shadeer/M1/ai-api/DataSources/Final_for_smart_pricing.csv'
raw_data = pd.read_csv(file_path, encoding='latin1')
raw_data["UID"]
raw_data["Original Order Value"]

0        5.36
1       33.80
2       52.21
3        4.50
4        9.98
        ...  
7691     9.09
7692    29.80
7693    11.22
7694    13.44
7695     7.42
Name: Original Order Value, Length: 7696, dtype: float64

### ABC anlaysis

In [4]:
import pandas as pd


data = pd.read_csv(file_path, encoding='ISO-8859-1')

# Drop rows where 'Order Value' is missing
data_cleaned = data.dropna(subset=['Order Value'])

# Calculate total order value per customer
customer_order_totals = data_cleaned.groupby('UID')['Order Value'].sum()

# Sort customers based on total order value
customer_order_totals_sorted = customer_order_totals.sort_values(ascending=False)
data_cleaned = data_cleaned.drop_duplicates(subset='UID')
data_cleaned["total_spend"] = customer_order_totals.values

df = data_cleaned
df['ABC_Category'] = pd.qcut(df['total_spend'], [0, 0.5, 0.8, 1], labels=['C', 'B', 'A'], duplicates='drop')

df['is_premium'] = df['ABC_Category'].isin(['B', 'A'])


In [5]:
# df["ABC_Category"].value_counts() 

In [6]:

# df['is_premium'] = df['ABC_Category'].isin(['B', 'A'])
# df['is_premium'].value_counts()



## Budget Allocation

In [4]:
# Main function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings("ignore")


class BudgetAllocation:
    def __init__(self, total_budget, base_budget_percent, premium_budget_percent, abc_split):
        self.total_budget = total_budget
        self.base_percent = base_budget_percent
        self.premium_percent = premium_budget_percent
        self.abc_split = abc_split
        self.df_path = "/Users/shadeer/Desktop/HOME/AI/PulseID/Notebook-instance/Shadeer/M1/ai-api/DataSources/Final_for_smart_pricing.csv"
        self.customers = self.ABC_analysis_df()
        self.redeemed_reward = 0

    def ABC_analysis_df(self):    
        data = pd.read_csv(self.df_path, encoding='ISO-8859-1')

        # Drop rows where 'Order Value' is missing
        data_cleaned = data.dropna(subset=['Order Value'])

        # Calculate total order value per customer
        customer_order_totals = data_cleaned.groupby('UID')['Order Value'].sum()

        # Sort customers based on total order value
        customer_order_totals_sorted = customer_order_totals.sort_values(ascending=False)
        data_cleaned = data_cleaned.drop_duplicates(subset='UID')
        data_cleaned["total_spend"] = customer_order_totals.values

        df = data_cleaned
        abc = self.abc_split
        # abc = [0, 0.8, 0.9, 1]
        df['ABC_Category'] = pd.qcut(df['total_spend'], abc, labels=['C', 'B', 'A'], duplicates='drop')

        df['is_premium'] = df['ABC_Category'].isin(['B', 'A'])
        return df
        
    def calculate_rewards(self):
        base_budget = self.total_budget * self.base_percent
        premium_budget = self.total_budget * self.premium_percent
        num_customers = len(self.customers)

        self.customers['base_reward'] = base_budget / num_customers
        premium_customers = self.customers['is_premium'].sum()
        self.customers['premium_reward'] = 0
        self.customers.loc[self.customers['is_premium'], 'premium_reward'] = premium_budget / premium_customers
        self.customers['total_reward'] = self.customers['base_reward'] + self.customers['premium_reward']

    def plot_rewards(self):
        plt.figure(figsize=(10, 6))
        plt.hist(self.customers['total_reward'], bins=30, alpha=0.7, label='Total Reward')
        plt.hist(self.customers['base_reward'], bins=30, alpha=0.7, label='Base Reward', color='r')
        plt.title('Distribution of Rewards')
        plt.xlabel('Reward Amount ($)')
        plt.ylabel('Number of Customers')
        plt.legend()
        plt.show()
    
    def summary(self):
        max_base_reward = self.customers['base_reward'].max()
        max_premium_reward = self.customers[self.customers['is_premium']]['premium_reward'].max()
        total_reward = max_base_reward + max_premium_reward
        print(f"Total Budget: ${self.total_budget}")
        print(f"Base Budget: ${self.total_budget * self.base_percent}")
        print(f"Premium Budget: ${self.total_budget * self.premium_percent}")
        print("--------------------------------------------")
        print(f"Number of Premium Customers: {self.customers['is_premium'].sum()}")
        print(f"Number of Total Unique Customers: {len(self.customers)}")
        print("--------------------------------------------")

        print(f"Average Base Reward: ${self.customers['base_reward'].mean():.2f}")
        print(f"Average Extra value for Premium Customer: ${self.customers[self.customers['is_premium']]['premium_reward'].mean():.2f}")
        print(f"Total Premium Reward: ${total_reward}")
        

        max_base_reward = self.customers['base_reward'].max()
        max_premium_reward = self.customers[self.customers['is_premium']]['premium_reward'].max()
        total_reward = max_base_reward + max_premium_reward

        summary_dict = {
            "Total Budget": f"${self.total_budget}",
            "Base Budget": f"${self.total_budget * self.base_percent}",
            "Premium Budget": f"${self.total_budget * self.premium_percent}",
            "Number of Premium Customers": self.customers['is_premium'].sum(),
            "Number of Total Unique Customers": len(self.customers),
            "Average Base Reward": f"${self.customers['base_reward'].mean():.2f}",
            "Average Extra value for Premium Customer": f"${self.customers[self.customers['is_premium']]['premium_reward'].mean():.2f}",
            "Total Premium Reward": f"${total_reward}"
        }
        
        # print("TOAL BUDGET IS: ", self.total_budget)

        return summary_dict
        # print("--------------------------------------------")
        # print(self.customers['is_premium'].value_counts())
        # print("--------------------------------------------")
        # print(self.customers['ABC_Category'].value_counts())





BA = BudgetAllocation( total_budget=2000, base_budget_percent=0.7,
                                 premium_budget_percent=0.3, abc_split=[0, 0.5, 0.8, 1])
BA.calculate_rewards()
BA.summary()

Total Budget: $2000
Base Budget: $1400.0
Premium Budget: $600.0
--------------------------------------------
Number of Premium Customers: 656
Number of Total Unique Customers: 1312
--------------------------------------------
Average Base Reward: $1.07
Average Extra value for Premium Customer: $0.91
Total Premium Reward: $1.981707317073171


{'Total Budget': '$2000',
 'Base Budget': '$1400.0',
 'Premium Budget': '$600.0',
 'Number of Premium Customers': 656,
 'Number of Total Unique Customers': 1312,
 'Average Base Reward': '$1.07',
 'Average Extra value for Premium Customer': '$0.91',
 'Total Premium Reward': '$1.981707317073171'}

## Commision


In [11]:
class BudgetAllocation:
    def __init__(self, total_budget, base_budget_percent, premium_budget_percent, abc_split, commission_type_1, commission_type_2):
        self.total_budget = total_budget
        self.base_percent = base_budget_percent
        self.premium_percent = premium_budget_percent
        self.abc_split = abc_split
        self.commission_1 = commission_type_1
        self.commission_2 = commission_type_2
        self.df_path = '/Users/shadeer/Desktop/HOME/AI/PulseID/Notebook-instance/Shadeer/M1/ai-api/DataSources/Final_for_smart_pricing.csv'
        self.customers = self.ABC_analysis_df()
        self.redeemed_reward = 0

    def ABC_analysis_df(self):
        data = pd.read_csv(self.df_path, encoding='ISO-8859-1')
        data_cleaned = data.dropna(subset=['Order Value'])
        customer_order_totals = data_cleaned.groupby('UID')['Order Value'].sum()
        data_cleaned = data_cleaned.drop_duplicates(subset='UID')
        data_cleaned["total_spend"] = customer_order_totals.values
        df = data_cleaned
        df['ABC_Category'] = pd.qcut(df['total_spend'], self.abc_split, labels=['C', 'B', 'A'], duplicates='drop')
        df['is_premium'] = df['ABC_Category'].isin(['B', 'A'])
        return df
    
    def calculate_rewards(self):
        base_budget = self.total_budget * self.base_percent
        premium_budget = self.total_budget * self.premium_percent
        num_customers = len(self.customers)

        # Calculate original rewards before commission
        self.customers['original_base_reward'] = base_budget / num_customers
        premium_customers = self.customers['is_premium'].sum()

        self.customers['original_premium_reward'] = 0
        if premium_customers > 0:
            self.customers.loc[self.customers['is_premium'], 'original_premium_reward'] = premium_budget / premium_customers

        # Calculate the total premium reward before applying any commission
        self.customers['base_reward'] = self.customers['original_base_reward']  # Using original base reward without commission
        self.customers['premium_reward'] = 0
        if premium_customers > 0:
            total_premium_reward = self.customers.loc[self.customers['is_premium'], 'original_premium_reward'] + self.customers.loc[self.customers['is_premium'], 'original_base_reward']
            # Apply both commissions to the total premium reward
            self.customers.loc[self.customers['is_premium'], 'premium_reward'] = total_premium_reward * (1 - self.commission_1 - self.commission_2)

        self.customers['total_reward'] = self.customers['base_reward'] + self.customers['premium_reward']
        self.total_base_commission = self.customers['original_base_reward'].sum() * self.commission_1
        self.total_premium_commission = total_premium_reward.sum() * (self.commission_1 + self.commission_2) if premium_customers > 0 else 0




    def plot_rewards(self):
        plt.figure(figsize=(10, 6))
        plt.hist(self.customers['total_reward'], bins=30, alpha=0.7, label='Total Reward')
        plt.hist(self.customers['base_reward'], bins=30, alpha=0.7, label='Base Reward', color='r')
        plt.title('Distribution of Rewards')
        plt.xlabel('Reward Amount ($)')
        plt.ylabel('Number of Customers')
        plt.legend()
        plt.show()

    def summary(self):
        print(f"Total Budget: ${self.total_budget:.2f}")
        print(f"Base Budget: ${self.total_budget * self.base_percent:.2f}")
        print(f"Premium Budget: ${self.total_budget * self.premium_percent:.2f}")
        print("---------------------------------------------------")
        print(f"Number of Premium Customers: {self.customers['is_premium'].sum()}")
        print(f"Number of Total Unique Customers: {len(self.customers)}")
        print("---------------------------------------------------")
        print(f"Base Reward: ${self.customers['original_base_reward'].mean():.2f}")
        print(f"Reduced Commission from Base Reward : ${self.customers['original_base_reward'].mean() * (self.commission_1 + self.commission_2) :.2f}")

        print(f"Base Reward Used for Premium Calculation: ${self.customers.loc[self.customers['is_premium'], 'original_base_reward'].mean() if self.customers['is_premium'].sum() > 0 else 0:.2f}")

        # Enhanced output for premium rewards
        if 'original_premium_reward' in self.customers.columns and self.customers['is_premium'].sum() > 0:
            print("Original Average Premium Reward (before adding base): ${:.2f}".format(
                self.customers.loc[self.customers['is_premium'], 'original_premium_reward'].mean()))
            print("Total Average Premium Reward (before commission): ${:.2f}".format(
                (self.customers.loc[self.customers['is_premium'], 'original_premium_reward'] + self.customers.loc[self.customers['is_premium'], 'original_base_reward']).mean()))
            print("Final Average Premium Reward (after commission): ${:.2f}".format(
                self.customers.loc[self.customers['is_premium'], 'premium_reward'].mean()))
        else:
            print("Original Average Premium Reward: $0.00")
            print("Total Average Premium Reward (before commission): $0.00")
            print("Final Average Premium Reward (after commission): $0.00")

        print("---------------------------------------------------")
        print(f"Total Commission Deducted from Base Rewards: ${self.total_base_commission:.2f}")
        print(f"Total Commission Deducted from Premium Rewards: ${self.total_premium_commission:.2f}")
        print(f"Total Commission Deducted: ${self.total_base_commission + self.total_premium_commission:.2f}")
        print("---------------------------------------------------")
        
    def detailed_summary(self):
        original_base_reward_mean = self.customers['original_base_reward'].mean()
        base_reward_after_commissions = original_base_reward_mean * (1 - self.commission_1) * (1 - self.commission_2)
        
        # Premium Reward details
        premium_customers = self.customers[self.customers['is_premium']]
        original_premium_reward_mean = premium_customers['original_premium_reward'].mean()
        original_base_reward_mean = premium_customers['original_base_reward'].mean()
        pulse_commission = self.commission_1
        partner_commission = self.commission_2

        # Calculate the Pulse commission on the premium reward
        pulse_commission_on_premium = ( original_premium_reward_mean + original_base_reward_mean )  * pulse_commission

        # Calculate the Partner commission on the premium reward
        partner_commission_on_premium = ( original_premium_reward_mean + original_base_reward_mean ) * partner_commission

        # Calculate the final premium reward after deducting both commissions
        final_premium_reward_after_commissions =  ( original_premium_reward_mean + original_base_reward_mean ) * (1 - pulse_commission) * (1 - partner_commission)

        print(f"Total Budget: ${self.total_budget:.2f}")
        print(f"Base Budget: ${self.total_budget * self.base_percent:.2f} ({self.base_percent * 100}% of total)")
        print(f"Premium Budget: ${self.total_budget * self.premium_percent:.2f} ({self.premium_percent * 100 } % of total)")
        print("---------------------------------------------------")
        print(f"Number of Total Unique Customers: {len(self.customers)}")
        print(f"Number of Premium Customers: {self.customers['is_premium'].sum()}")
        print("---------------------------------------------------")

        # Base Reward details
        print("Base Reward Details:")
        # (before Pulse Commission)
        print(f"  - Base Reward : ${self.customers['original_base_reward'].mean():.2f}")
        print(f"  - Pulse Commission on Base Reward: ${self.customers['original_base_reward'].mean() * self.commission_1:.2f} (Pulse Commission: {self.commission_1*100}%)")
        print(f"  - Partner Commission on Base Reward: ${self.customers['original_base_reward'].mean() * self.commission_2:.2f} (Partner Commission: {self.commission_2*100}%)")
        # print(f"  - Base Reward after Pulse and Partner Commission: ${self.customers['base_reward'].mean():.2f}")
        print(f"  - Base Reward after Pulse and Partner Commission: ${base_reward_after_commissions:.2f}")

        print("---------------------------------------------------")

        # Premium Reward details
        if 'original_premium_reward' in self.customers.columns and self.customers['is_premium'].sum() > 0:
            # Print the details
            print("Premium Reward Details:")
            print(f"  - Extra Premium Reward: ${original_premium_reward_mean:.2f}")
            print(f"  - Total Premium Reward ( + Base ): ${original_premium_reward_mean + original_base_reward_mean:.2f}")
            print(f"  - Pulse Commission on Premium Reward: ${pulse_commission_on_premium:.2f} (Pulse Commission: {pulse_commission * 100:.2f}%)")
            print(f"  - Partner Commission on Premium Reward: ${partner_commission_on_premium:.2f} (Partner Commission: {partner_commission * 100:.2f}%)")
            print(f"  - Final Premium Reward (after Pulse and Partner Commissions): ${final_premium_reward_after_commissions:.2f}")
        else:
            print("Premium Reward Details:")
            print("  - No premium rewards calculated (No premium customers identified).")

        print("---------------------------------------------------")
        # Commission Summary
        # print(f"  - From Base Rewards (Pulse Commission): ${ (self.total_budget * self.base_percent ) * self.commission_1:.2f}")
        # print(f"  - From Base Rewards (Partner Commission): ${ (self.total_budget * self.base_percent ) * self.commission_2:.2f}")

        # print(f"  - From Premium Rewards (Pulse Commission): ${ (self.total_budget * self.premium_percent ) * self.commission_1:.2f}")
        # print(f"  - From Premium Rewards (Partner Commission): ${ (self.total_budget * self.premium_percent ) * self.commission_2:.2f}")
 
        # # print(f"  - From Premium Rewards (Pulse Commission): ${self.total_budget * self.commission_2:.2f}")
        # # print(f"  - From Base Rewards (Pulse Commission): ${self.total_base_commission:.2f}")
        # # print(f"  - From Premiumn Rewards (Pulse Commission): ${self.total_base_commission:.2f}")

        # print(f"  - Total Commission Deducted: ${self.total_base_commission + self.total_premium_commission:.2f}")
        
        pulse_base_reward = (self.total_budget * self.base_percent) * self.commission_1
        partner_base_reward = (self.total_budget * self.base_percent) * self.commission_2
        pulse_premium_reward = (self.total_budget * self.premium_percent) * self.commission_1
        partner_premium_reward = (self.total_budget * self.premium_percent) * self.commission_2

        # Print individual commission details
        print("Total Commissions Deducted:")
        print(f"  - From Base Rewards (Pulse Commission): ${pulse_base_reward:.2f}")
        print(f"  - From Base Rewards (Partner Commission): ${partner_base_reward:.2f}")
        print(f"  - From Premium Rewards (Pulse Commission): ${pulse_premium_reward:.2f}")
        print(f"  - From Premium Rewards (Partner Commission): ${partner_premium_reward:.2f}")

        # Calculate and print the total commission
        total_commission = pulse_base_reward + partner_base_reward + pulse_premium_reward + partner_premium_reward




ba = BudgetAllocation(
    total_budget=6000,
    base_budget_percent=0.7,
    premium_budget_percent=0.3,
    abc_split=[0, 0.5, 0.8, 1],
    commission_type_1=0.15, 
    commission_type_2=0.05  
)

ba.calculate_rewards()
ba.detailed_summary()


Total Budget: $4000.00
Base Budget: $2800.00 (70.0% of total)
Premium Budget: $1200.00 (30.0 % of total)
---------------------------------------------------
Number of Total Unique Customers: 1312
Number of Premium Customers: 656
---------------------------------------------------
Base Reward Details:
  - Base Reward : $2.13
  - Pulse Commission on Base Reward: $0.32 (Pulse Commission: 15.0%)
  - Partner Commission on Base Reward: $0.11 (Partner Commission: 5.0%)
  - Base Reward after Pulse and Partner Commission: $1.72
---------------------------------------------------
Premium Reward Details:
  - Extra Premium Reward: $1.83
  - Total Premium Reward ( + Base ): $3.96
  - Pulse Commission on Premium Reward: $0.59 (Pulse Commission: 15.00%)
  - Partner Commission on Premium Reward: $0.20 (Partner Commission: 5.00%)
  - Final Premium Reward (after Pulse and Partner Commissions): $3.20
---------------------------------------------------
Total Commissions Deducted:
  - From Base Rewards (Pu

## Redemption

In [ ]:
  
  
    # def redemption(self, minimum_spent, reward_premium, base_reward):
    #     import pandas as pd
    #     df = self.ABC_analysis_df()
    #     order_value = df['Order Value']
        
    #     is_prem = df[df['is_premium']== True].shape[0]
    #     is_prem_claimed = df[(df['is_premium']== True) & (order_value > minimum_spent)].shape[0]
    #     is_base_claimed = df[(df['is_premium']== False) & (order_value > minimum_spent)].shape[0]

    #     total_reward = (is_prem_claimed * reward_premium) + (is_base_claimed * base_reward)
    #     self.redeemed_amount += total_reward
    #     self.total_budget -= self.redeemed_amount  # Update the total budget

    #     print(f"Redemption Summary:")
    #     print(f"Total Premium Customers: {is_prem}")
    #     print(f"Premium Customers Claimed: {is_prem_claimed}")
    #     print(f"Base Customers Claimed: {is_base_claimed}")
    #     print(f"Total Reward: {total_reward}")
    #     print(f"Remaining Budget: {self.total_budget}")
    #     print(f"---------------------------------------")

In [24]:
# minimum_spent = 20
# budget = 2000
# reduced_budget = 1200


# BA = BudgetAllocation( total_budget=reduced_budget, base_budget_percent=0.7, premium_budget_percent=0.3, abc_split=[0, 0.5, 0.8, 1])
# BA.calculate_rewards()
# BA.summary()


In [48]:
# df = BA.ABC_analysis_df()

# # budget = 2000
# minimum_spent = 20
# reward_premium = 1.981
# base_reward = 1.07
# budget = 2000

# def redemption():
#     df = BA.ABC_analysis_df()
#     order_value = df['Order Value']
    
#     is_prem = df[df['is_premium']== True].shape[0]
#     is_prem_claimed = df[(df['is_premium']== True) & (order_value > minimum_spent)].shape[0]
#     is_base_claimed = df[(df['is_premium']== False) & (order_value > minimum_spent)].shape[0]

#     total_reward = (is_prem_claimed * reward_premium) + (is_base_claimed * base_reward)
#     print(f"Redemption Summary:")
#     print(f"Total Premium Customers: {is_prem}")
#     print(f"Premium Customers Claimed: {is_prem_claimed}")
#     print(f"Base Customers Claimed: {is_base_claimed}")
#     print(f"Total Reward: {total_reward}")
#     print(f"---------------------------------------")
#     return is_prem, is_prem_claimed, is_base_claimed, total_reward
    
    
# is_prem, is_prem_claimed, is_base_claimed, total_reward = redemption()

# new_budget = budget - total_reward
# BA = BudgetAllocation( total_budget=new_budget, base_budget_percent=0.7,
#                                  premium_budget_percent=0.3, abc_split=[0, 0.5, 0.8, 1])

# BA.calculate_rewards()
# BA.summary()
# print("____________REDEMPTION)__)_______________________")

# BA.redemption2(minimum_spent, reward_premium, base_reward)

Redemption Summary:
Total Premium Customers: 656
Premium Customers Claimed: 204
Base Customers Claimed: 207
Total Reward: 625.614
---------------------------------------
Total Budget: $1374.386
Base Budget: $962.0701999999999
Premium Budget: $412.31579999999997
--------------------------------------------
Number of Premium Customers: 656
Number of Total Unique Customers: 1312
--------------------------------------------
Base Reward: $0.73
Extra value for Premium Customer: $0.63
Total Premium Reward: $1.3618153963414632
____________REDEMPTION)__)_______________________
Redemption Summary:
Total Premium Customers: 656
Premium Customers Claimed: 204
Base Customers Claimed: 207
Total Reward: 625.614
---------------------------------------


(656, 204, 207, 625.614)

In [82]:
INITIAL_BUDGET = 2000
BA = BudgetAllocation( total_budget=INITIAL_BUDGET, base_budget_percent=0.7,
                                 premium_budget_percent=0.3, abc_split=[0, 0.5, 0.8, 1])

BA.calculate_rewards()
BA.summary()

Total Budget: $2000
Base Budget: $1400.0
Premium Budget: $600.0
--------------------------------------------
Number of Premium Customers: 656
Number of Total Unique Customers: 1312
--------------------------------------------
Base Reward: $1.07
Extra value for Premium Customer: $0.91
Total Premium Reward: $1.981707317073171


In [83]:
base_reward = 1.07
reward_premium = 1.981

In [84]:
# First round
minimum_spent = 40
BA.redemption2(minimum_spent, reward_premium, base_reward)
print("total redeemed reward", BA.redeemed_reward )
print("remaining budget", BA.total_budget - BA.redeemed_reward )

Redemption Summary:
Total Premium Customers: 656
Premium Customers Claimed: 106
Base Customers Claimed: 113
Total Reward: 330.896
---------------------------------------
total redeemed reward 330.896
remaining budget 1669.104


In [85]:
# second round
minimum_spent = 40
BA.redemption2(minimum_spent, reward_premium, base_reward)
print("total redeemed reward", BA.redeemed_reward )
print("remaining budget", BA.total_budget - BA.redeemed_reward )

Redemption Summary:
Total Premium Customers: 656
Premium Customers Claimed: 106
Base Customers Claimed: 113
Total Reward: 330.896
---------------------------------------
total redeemed reward 661.792
remaining budget 1338.208


In [86]:
# Third round
minimum_spent = 40
BA.redemption2(minimum_spent, reward_premium, base_reward)
print("total redeemed reward", BA.redeemed_reward )
print("remaining budget", BA.total_budget - BA.redeemed_reward )

Redemption Summary:
Total Premium Customers: 656
Premium Customers Claimed: 106
Base Customers Claimed: 113
Total Reward: 330.896
---------------------------------------
total redeemed reward 992.6880000000001
remaining budget 1007.3119999999999


In [47]:
is_prem, is_prem_claimed, is_base_claimed, total_reward = redemption()

new_budget = budget - total_reward
BA = BudgetAllocation( total_budget=new_budget, base_budget_percent=0.7,
                                 premium_budget_percent=0.3, abc_split=[0, 0.5, 0.8, 1])

BA.calculate_rewards()
BA.summary()
BA.redemption2(minimum_spent, reward_premium, base_reward)

Redemption Summary:
Total Premium Customers: 656
Premium Customers Claimed: 65
Base Customers Claimed: 75
Total Reward: 209.01500000000001
---------------------------------------
Total Budget: $1790.985
Base Budget: $1253.6895
Premium Budget: $537.2955
--------------------------------------------
Number of Premium Customers: 656
Number of Total Unique Customers: 1312
--------------------------------------------
Base Reward: $0.96
Extra value for Premium Customer: $0.82
Total Premium Reward: $1.7746040396341463
Redemption Summary:
Total Premium Customers: 656
Premium Customers Claimed: 65
Base Customers Claimed: 75
Total Reward: 209.01500000000001
---------------------------------------


(656, 65, 75, 209.01500000000001)

### Plotting Minimums with Redeeming value

In [12]:
# Plotting Redemption

import pandas as pd
import plotly.express as px

# Assuming BA.ABC_analysis_df() returns your DataFrame
df = BA.ABC_analysis_df()

reward_premium = 1.981
base_reward = 1.07

def compute_reward(min_spent):
    order_value = df['Order Value']
    
    is_prem_claimed = df[(df['is_premium'] == True) & (order_value > min_spent)].shape[0]
    is_base_claimed = df[(df['is_premium'] == False) & (order_value > min_spent)].shape[0]

    total_reward = (is_prem_claimed * reward_premium) + (is_base_claimed * base_reward)
    
    return total_reward, is_prem_claimed, is_base_claimed

# Simulate different minimum spent values
min_spent_values = range(0, 500, 10)  # Adjust the range and step as needed
results = [compute_reward(min_spent) for min_spent in min_spent_values]

# Create a DataFrame to store the results and additional data for hovering
results_df = pd.DataFrame({
    'Minimum Spent': min_spent_values,
    'Total Reward': [result[0] for result in results],
    'Premium Claims': [result[1] for result in results],
    'Base Claims': [result[2] for result in results]
})

# Create an interactive plot
fig = px.line(results_df, x='Minimum Spent', y='Total Reward', 
              title='Impact of Minimum Spent on Total Rewards',
              labels={'Total Reward': 'Total Reward ($)', 'Minimum Spent': 'Minimum Spent ($)'})

# Add hover data
fig.update_traces(mode='lines+markers', hoverinfo='text')
fig.update_traces(text=["Premium Claims: {}<br>Base Claims: {}".format(p, b) for p, b in zip(results_df['Premium Claims'], results_df['Base Claims'])])

fig.show()


In [41]:
import random

# Initialize the BudgetAllocation with an initial budget
BA = BudgetAllocation(total_budget=2000, base_budget_percent=0.7, premium_budget_percent=0.3, abc_split=[0, 0.5, 0.8, 1])

minimum_spent = 20
# Initial budget
print("Initial Total Budget:", BA.total_budget)

    # Perform redemption
BA.redemption(minimum_spent, reward_premium, base_reward)
    
    # Print remaining budget

Initial Total Budget: 2000
Redemption Summary:
Total Premium Customers: 656
Premium Customers Claimed: 204
Base Customers Claimed: 207
Total Reward: 4929
Remaining Budget: -2929
---------------------------------------


In [36]:
BA = BudgetAllocation(total_budget=2000, base_budget_percent=0.7, premium_budget_percent=0.3, abc_split=[0, 0.5, 0.8, 1])
BA.redemption(minimum_spent=20, reward_premium=1.981, base_reward=1.07)


0

In [37]:
BA = BudgetAllocation(total_budget=2000, base_budget_percent=0.7, premium_budget_percent=0.3, abc_split=[0, 0.5, 0.8, 1])
BA.calculate_rewards()
BA.summary()

Total Budget: $2000
Base Budget: $1400.0
Premium Budget: $600.0
--------------------------------------------
Number of Premium Customers: 656
Number of Total Unique Customers: 1312
--------------------------------------------
Base Reward: $1.07
Extra value for Premium Customer: $0.91
Total Premium Reward: $1.981707317073171


In [38]:
BA.redemption(minimum_spent=20, reward_premium=1.981, base_reward=1.07)




Redemption Summary:
Total Premium Customers: 656
Premium Customers Claimed: 204
Base Customers Claimed: 207
Total Reward: 625.614
Remaining Budget: 1374.386
---------------------------------------


In [39]:
BA.summary()

Total Budget: $1374.386
Base Budget: $962.0701999999999
Premium Budget: $412.31579999999997
--------------------------------------------
Number of Premium Customers: 656
Number of Total Unique Customers: 1312
--------------------------------------------
Base Reward: $1.07
Extra value for Premium Customer: $0.91
Total Premium Reward: $1.981707317073171


In [40]:
BA.redemption(minimum_spent=20, reward_premium=1.981, base_reward=1.07)

Redemption Summary:
Total Premium Customers: 656
Premium Customers Claimed: 204
Base Customers Claimed: 207
Total Reward: 625.614
Remaining Budget: 123.1579999999999
---------------------------------------
